In [43]:
#########################################################################################################################
# MAD2502 Capstone Project: Equity-Index Derivatives Weighting, Arbitrage, and Risk-Management Dashboard (EDWARD)       #
#                                                                                                                       #
# Group: 20                                                                                                             #
# Members: J. Ang (jasonang@ufl.edu), E. Bradford (ezzard.bradford@ufl.edu), and D. Harrison (d.harrison1@ufl.edu)      #
#                                                                                                                       #
# Permission given to modify the code as long as you keep this declaration at the top                                   #
#########################################################################################################################

import datetime as dt 
from datetime import datetime,timedelta
import numpy as np  
import pandas as pd
import pandas_datareader as pdr
import matplotlib.pyplot as plt
import yahoo_fin.stock_info as si
import yfinance as yf 


class Edward:
    
    def __init__(self, tolerance: str, expiry: str, liquid: float):
              
        self.index_components = pd.read_csv('nasdaq100_components.csv', usecols=[0,1])

        self.tolerance = float(tolerance[:-1])/100
        
        if(expiry[0:3] == "MAR"):
            expiry = expiry[-4:] + "-03-01"
            
        elif(expiry[0:3] == "JUN"):
            expiry = expiry[-4:] + "-06-01"
            
        elif(expiry[0:3] == "SEP"):
            expiry = expiry[-4:] + "-09-01"
            
        elif(expiry[0:3] == "DEC"):
            expiry = expiry[-4:] + "-12-01"
            
        self.today = f"{datetime.now().year}-{datetime.now().month}-{datetime.now().day}"
        self.t0 = datetime.strptime(self.today, "%Y-%m-%d")
        self.tf = datetime.strptime(expiry, "%Y-%m-%d")
        self.dt = (7 + (self.tf - self.t0).days) / 365
        
        assert self.dt <= 2
        
        self.n_c = liquid
    
    def r_f(self):
        
        inflation_rates = pd.concat(map(pd.read_csv, [
            'QCPI20224.csv', 'QCPI20223.csv', 'QCPI20222.csv', 'QCPI20221.csv', 
            'QCPI20214.csv', 'QCPI20213.csv','QCPI20212.csv', 'QCPI20211.csv',
            ]))
        
        inflation = inflation_rates['CPI Inflation'].mean() / 100
                        
        treaury_rates = pd.concat(map(pd.read_csv, [
            'rates2022.csv', 'rates2021.csv', 'rates2020.csv', 'rates2019.csv', 
            'rates2018.csv', 'rates2017.csv',
            ]), ignore_index=True)
                          
        treasury_yield = treaury_rates["3 Mo"].mean()

        if 3 / 12 <= self.dt < 1 / 2:
            treasury_yield = treaury_rates["6 Mo"].mean()
            
        elif 1 / 2 <= self.dt < 3 / 4:
            treasury_yield = treaury_rates["1 Yr"].mean()
            
        elif 3 / 4 <= self.dt < 1:
            treasury_yield = treaury_rates["1 Yr"].mean()
            
        elif 1 <= self.dt < 5 / 4:
            treasury_yield = treaury_rates["2 Yr"].mean()
            
        elif 5 / 4 <= self.dt < 3 / 2:
            treasury_yield = treaury_rates["2 Yr"].mean()
            
        elif 3 / 2 <= self.dt < 2:
            treasury_yield = treaury_rates["2 Yr"].mean()
            
        return treasury_yield - inflation
    
    def equity_ticker(self,ticker: str):
        
        return yf.Ticker(str(ticker).upper())
    
    def equity_price(self,ticker: str):
        
        return self.equity_ticker(ticker).info['regularMarketPrice']
    
    def equity_yield(self,ticker: str):
        
        if len(self.equity_ticker(ticker).dividends) != 0:
            
            return 4 * np.mean(self.equity_ticker(ticker).dividends) / self.equity_price(ticker)
        
        return 0 
    
    def nasdaq100_components(self):
        
        nasdaq100_components = []
        
        for component in self.index_components["Symbol"]:
            
            nasdaq100_components.append(component)            
        
        return nasdaq100_components
    
    def nasdaq100_weights(self):
        
        component_weights = []
        
        component_caps = []
        
        for component in self.nasdaq100_components():
            
            component_cap_str = si.get_quote_table(component)["Market Cap"]
                        
            if(str(component_cap_str)[-1] == "T"):
                component_caps.append(float(str(component_cap_str)[:-1]) * 10 ** 12)
                    
            elif(str(component_cap_str)[-1] == "B"):
                component_caps.append(float(str(component_cap_str)[:-1]) * 10 ** 9)
                
        nasdaq100_cap = np.sum(component_caps)
            
        for cap in component_caps:
            
            component_weights.append(component_caps / nasdaq100_cap)
            
        return component_caps, nasdaq100_cap, component_weights
    
    
    def n_s(self,constraint: float):
        
        n_s = []
        
        Q_s = []
        
        for i in range(len(self.nasdaq100_weights()[2]) + 1):
            
            n_s.append(constraint * self.n_c * self.nasdaq100_weights()[2][i] / self.equity_price(self.nasdaq100_components()[i]))
            Q_s.append(n_s[i] * self.equity_price(self.nasdaq100_components()[i]))
        
        return n_s, np.sum(Q_s)
    
    def nasdaq100_yield(self):
        
        nasdaq100_yield = []
        
        for i in range(len(self.nasdaq100_components()) + 1):
            
            nasdaq100_yield.append(self.nasdaq100_weights()[2][i] * self.equity_yield(self.nasdaq100_components()[i]))
        
        return np.sum(nasdaq100_yield)
    
    def settlements(self,asset: str):
        tempCsv1 = f"{self.equity_ticker(asset)}.csv"
        tempCsv2 = self.equity_ticker(asset).history(period = "2y").to_csv(tempCsv1)
        settlements = pd.read_csv(tempCsv1)[["Date","Close"]]
        return settlements
    
    def r(self,asset: str):
        
        r = []
        
        for i in range(len(self.settlements(asset))):
            
            r.append(self.settlements(asset)[i + 1] / self.settlements(asset)[i] - 1)
            
        up = 0
            
        for movement in r:
            
            if movement > 0:
                
                up += 1
                
        p_rise = up / (252 * 2)               
        
        return r, np.mean(r), np.std(r), p_rise, np.std(r) ** 2
    
    def equity_performance(self):
        
        mean_r_s = []
        
        expected_r_s = []
        
        std_s = []
        
        for i in range(len(self.nasdaq100_components()) + 1):
            
            mean_r_s.append(self.nasdaq100_weights()[2][i] * self.r(self.nasdaq100_components()[i])[0][i])
            
            expected_r_s.append(self.r(self.nasdaq100_components()[i])[3][i] * self.nasdaq100_weights()[2][i] * self.r(self.nasdaq100_components()[i])[0][i])
            
            std_s.append(self.nasdaq100_weights()[2][i] * self.r(self.nasdaq100_components()[i])[2][i])
        
        return np.sum(mean_r_s), np.sum(expected_r_s), np.sum(std_s)
    
    def beta(self,asset):
        
        beta = []
        
        beta_leg = []
        
        for i in range(len(self.nasdaq100_components()) + 1):
            
            beta.append(np.covar(self.r(asset,self.r("^NDX")[0][i]) / self.r("^NDX")[2][i]))
            
            beta_leg.append(self.nasdaq100_weights()[2][i] * beta[i])
    
        return beta, np.sum(beta_leg)
    
    def sharpe_s(self):
        
        alpha = self.equity_performance()[1] - self.equity_performance()[0]
        
        sharpe = alpha / self.equity_performance()[2]
        
        return alpha, sharpe
    
    def futures_price(self):
        
        return self.equity_price("^NDX") * np.e ** ((self.r_f() - self.nasdaq100_yield()) * self.dt)
    
    
    def r_futures(self):
        
        r_futures = []
        
        for i in range(len(self.settlements("QQQM"))):
            
            r_futures.append(self.settlements("QQQM")[i + 1] / self.settlements("QQQM")[i] - 1)
            
        up = 0
            
        for movement in r_futures:
            
            if movement > 0:
                
                up += 1
                
        p_futures_rise = up / (252 * 2)               
        
        return r_futures, np.mean(r_futures), np.std(r_futures), p_futures_rise, np.std(r_futures) ** 2 
    
    def ideal_futures_contracts(self):
        
        return self.beta(self.r_futures()) * self.n_s(0.5)[1] / 20 * self.futures_price()
    
    def futures_performance(self):
        
        mean_r_futures = []
        
        expected_r_futures = []
        
        std_futures = []
        
        for i in range(len(self.nasdaq100_components()) + 1):
            
            mean_r_futures.append(self.ideal_futures_contracts() * self.futures_price() * r_futures()[1] / self.n_c)
            
            expected_r_futures.append(self.r_futures()[3][i] * self.ideal_futures_contracts() * self.futures_price() * r_futures()[3] / self.n_c)
            
            std_futures.append(self.ideal_futures_contracts() * self.futures_price() * r_futures()[2] / self.n_c)
        
        return np.sum(mean_r_futures), np.sum(expected_r_futures), np.sum(std_futures)
                        
    def sharpe_futures(self):
                        
        alpha = self.futures_performance()[1] - self.futures_performance()[0]
        
        sharpe = alpha / self.futures_performance()[2]
        
        return alpha, sharpe
                        
    def signal(self):
        
        if(self.futures_price() < 100.8 * self.equity_price("QQQM")):
            
            return -self.n_s(0.5), self.ideal_futures_contracts(), (self.n_c + self.n_s()[1] - self.ideal_futures_contracts() * self.futures_price()) / self.equity_price("US2Y") 
                        
        elif(self.futures_price() > 100.8 * self.equity_price("QQQM")):
                        
            return self.n_s(0.5), -self.ideal_futures_contracts(), (self.n_c - self.n_s()[1] + self.ideal_futures_contracts() * self.futures_price()) / self.equity_price("US2Y") 
                        
        else:
                        
            return self.n_c / self.equity_price("US2Y")
    
    def portfolio(self):
        
        data = [("Future","NQ00",round(self.ideal_futures_contracts(),0),100.8 * self.equity_price("QQQM"),100.8 * self.settlements("QQQM")[-1]),
            ("Treasury","US2Y",round(self.signal()[2],0),self.equity_price("US2Y"),self.settlements("US2Y")[-1])
            ]
                        
        df = pd.DataFrame(data, columns = ["Instrument","Name","Qty","Price","Settlement"])
              
        for component in self.nasdaq100_components():
            df2 = {"Equity",component.upper(),self.n_s()[0][i],self.equity_price(component),self.settlements(component)[-1]}
            df = df.append(df2, ignore_index = True)
        
        return df
    
    def metrics(self):
        
        data = [(
            (self.n_s()[1] / self.n_c) * self.equity_performance()[2] + self.futures_performance()[2] * self.ideal_futures_contracts() * self.futures_price() / self.n_c - self.tolerance,
            (self.n_s()[1] / self.n_c) * self.sharpe_s()[0] + self.sharpe_futures()[0] * self.ideal_futures_contracts() * self.futures_price() / self.n_c,
            (self.n_s()[1] / self.n_c) * self.sharpe_s()[1] + self.sharpe_futures()[1] * self.ideal_futures_contracts() * self.futures_price() / self.n_c,
            (str(100*(self.n_s()[1] / self.n_c) * self.equity_performance()[0] + self.futures_performance()[0] * self.ideal_futures_contracts() * self.futures_price() / self.n_c)) + "%"
        )]              
                        
        df = pd.DataFrame(data, columns = ["Risk","Alpha","Sharpe","P/L"]) 